<h1><center>Предсказание стоимости жилья</center></h1>

# Введение

Задача обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели используем метрики RMSE, MAE и R2.

# Подготовка данных

In [2]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import Imputer

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml.feature import OneHotEncoder 
from pyspark.ml.feature import StringIndexer

RANDOM_SEED = 22

Инициализируем сессию spark и загрузим датасет

In [ ]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California Housing") \
                    .getOrCreate()

df = spark.read.option('header', 'true').csv('datasets/housing.csv', inferSchema = True) 

Посмотрим типы колонок

In [ ]:
df.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Посмотрим общую информацию о параметрах

In [ ]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Удалим дупликаты

In [ ]:
df = df.dropDuplicates()

Удалим записи, если в них встречаются пропуски в колонках долготы и ширины, т.к их не заменить соответствующими значениями

In [ ]:
unfix_subset = ['longitude', 'latitude']
df = df.na.drop(subset=unfix_subset)

В остальных колонках заменим на медианные значения

In [ ]:
fix_subset = ['housing_median_age', 'total_rooms', 'total_bedrooms',
                 'population', 'households', 'median_income']

imputer = Imputer(
    inputCols=fix_subset, 
    outputCols=fix_subset
    ).setStrategy("median")

df = imputer.fit(df).transform(df)

Выделим категориальные, количественные признаки и таргет

In [ ]:
categorical_cols = ['ocean_proximity']
numerical_cols   = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',
                   'population', 'households', 'median_income']
target = 'median_house_value'

кодируем категориальный параметр StringIndexer'ом

In [ ]:
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_idx")
df = indexer.fit(df).transform(df)

кодируем полученный параметр OneHotEncoder'ом

In [ ]:
ohe = OneHotEncoder(inputCol="ocean_proximity_idx", outputCol="ocean_proximity_ohe")
df = ohe.transform(df)

In [ ]:
df.select('ocean_proximity_ohe').toPandas()

,ocean_proximity_ohe
0,"(0.0, 0.0, 0.0, 1.0)"
1,"(0.0, 0.0, 0.0, 1.0)"
2,"(0.0, 0.0, 0.0, 1.0)"
3,"(0.0, 0.0, 0.0, 1.0)"
4,"(0.0, 1.0, 0.0, 0.0)"
...,...
20635,"(0.0, 1.0, 0.0, 0.0)"
20636,"(0.0, 1.0, 0.0, 0.0)"
20637,"(0.0, 1.0, 0.0, 0.0)"
20638,"(0.0, 0.0, 1.0, 0.0)"


соберем категориальные признаки с помощью VectorAssembler

In [ ]:
categorical_assembler = VectorAssembler(inputCols=['ocean_proximity_ohe'],
                                        outputCol='categorical_features')
df = categorical_assembler.transform(df) 

соберем количественные признаки с помощью VectorAssembler

In [ ]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                      outputCol="numerical_features")
df = numerical_assembler.transform(df) 

cкалирум количественные признаки с помощью StandartScaler

In [ ]:
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled",
                                withMean=False,
                                withStd=True)
df = standardScaler.fit(df).transform(df) 

Сделаем два набора признаков. Один с категориальными и числовыми признаками. Другой только с числовыми

In [ ]:
features_model_1 = ['categorical_features', 'numerical_features_scaled']
features_model_2 = ['numerical_features_scaled']

final_assembler_1 = VectorAssembler(inputCols=features_model_1, 
                                  outputCol="features_model_1") 
final_assembler_2 = VectorAssembler(inputCols=features_model_2, 
                                  outputCol="features_model_2") 

df = final_assembler_1.transform(df)
df = final_assembler_2.transform(df)

Разделим данные на тренировочный и тестовый сет

In [ ]:
train_data, test_data = df.randomSplit([0.75, 0.25], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

15527 5113


# Обучение моделей

Обучим модели линейной регрессии для каждого из наборов признаков

In [ ]:
lr_1 = LinearRegression(labelCol=target, featuresCol='features_model_1')
lr_2 = LinearRegression(labelCol=target, featuresCol='features_model_2')

model_lr_1 = lr_1.fit(train_data)
model_lr_2 = lr_2.fit(train_data)

Получим предсказания

In [ ]:
prediction_1 = model_lr_1.evaluate(test_data)
prediction_2 = model_lr_2.evaluate(test_data)

Сравним метрики обеих моделей

In [ ]:
print("RMSE model 1 and 2: " + str(round(prediction_1.rootMeanSquaredError, 1))
                             + " "
                             + str(round(prediction_2.rootMeanSquaredError, 1)))

RMSE model 1 and 2: 69096.2 69756.3


In [ ]:
print("R2 model 1 and 2: " + str(round(prediction_1.r2, 3))
                           + " "
                           + str(round(prediction_2.r2, 3)))

R2 model 1 and 2: 0.645 0.638


In [ ]:
print("MAE model 1 and 2: " + str(round(prediction_1.meanAbsoluteError, 1))
                            + " "
                            + str(round(prediction_2.meanAbsoluteError, 1)))

MAE model 1 and 2: 50288.0 51094.7


# Анализ результатов

По всем полученным метрикам модель с категориальными признаками показала себя чуть лучше, чем модель без них